In [1]:
# Install required packages
!pip install -qU transformers datasets wandb accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/arabic-history-questions-dataset/all_data_flattened.json
/kaggle/input/arabic-history-questions-dataset/all_data.json
/kaggle/input/arabic-history-questions-dataset/train.json
/kaggle/input/arabic-history-questions-dataset/test.json
/kaggle/input/arabic-history-questions-dataset/test_raw.json
/kaggle/input/arabic-history-questions-dataset/val_raw.json
/kaggle/input/arabic-history-questions-dataset/train_raw.json
/kaggle/input/arabic-history-questions-dataset/val.json


In [3]:
# Import libraries
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq
)
from datasets import DatasetDict, Dataset
# from huggingface_hub import notebook_login
import wandb

2025-05-11 18:23:47.280188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746987827.564333      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746987827.640370      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# # Login to Hugging Face Hub
# notebook_login()

wandb.login(key='77eafacbf29d3f89b810de78fe1f766a9b6e6fe8')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: noureenaboarab2003 (noureenaboarab2003-ain-shams-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
# Load dataset function
def load_data(train_path, val_path):
    print("🔄 Loading datasets...")
    train_data = pd.read_json(train_path)
    val_data = pd.read_json(val_path)
    
    train_data.rename(columns={"paragraph": "context"}, inplace=True)
    val_data.rename(columns={"paragraph": "context"}, inplace=True)
    
    print("\n📊 Training Data Sample:")
    print(train_data.head(1).T)
    print("\n📊 Validation Data Sample:")
    print(val_data.head(1).T)
    
    return train_data, val_data

train_path = "/kaggle/input/arabic-history-questions-dataset/train.json"
val_path = "/kaggle/input/arabic-history-questions-dataset/val.json"

train_data, val_data = load_data(train_path, val_path)

🔄 Loading datasets...

📊 Training Data Sample:
                                                          0
context   ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من ا...
question  ما هو الغرض الرئيسي من الممر الذي يتفرع إلى به...
answer                                   للاستقبال والترفيه

📊 Validation Data Sample:
                                                          0
context   كانت الحكومة المصرية على عهد محمد على حكومة مط...
question         ما الفرق بين حكومة محمد علي وعصر المماليك؟
answer                                 وجود نظام إداري منظم


In [6]:
# Format dataset
def format_for_qa(dataset, dataset_name="train"):
    print(f"\n🔍 Formatting {dataset_name} dataset...")
    formatted_text = dataset["question"] + " <sep> " + dataset["context"]
    
    print("\n📝 Sample Formatted Input (first 200 chars):")
    for i in range(3):
        print(f"Sample {i+1}: {formatted_text.iloc[i][:200]}...")
    
    return pd.DataFrame({
        "text": formatted_text,
        "answer": dataset["answer"]
    })

train_qa = format_for_qa(train_data, "train")
val_qa = format_for_qa(val_data, "validation")


🔍 Formatting train dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: ما هو الغرض الرئيسي من الممر الذي يتفرع إلى بهو في مساكن الطبقة الوسطى؟ <sep> ـ مساكن الطبقة الوسطى: كانت مخصصة للفنيين من الصناع، وكانت أكثر اتساعًا، ويشمل كل منزل فناء مكشوفًا وأربع حجرات أبوابها مف...
Sample 2: أين كانت تقع غرفة النوم في القصر الفرعوني؟ <sep> القصر الفرعونى: تميز القصر الفرعونى بالاتساع والفخامة، وكان يسمى بـ)البيت العظيم، ( وقُسم إلى قسمين: قسم عام يستقبل فيه الأشراف وكبار رجال الدولة، وقسم...
Sample 3: ما الذي جعل الحكومة التي أنشأها زايفنغ تُعرف باسم "مجلس الوزراء الملكي"؟ <sep> توفي إمبراطور قوانغشو في ١٤ نوفمبر ١٩٠٨ ، وفي ١٥ نوفمبر ١٩٠٨ ، توفي تسيشي أيضا . وزعمت الشائعات أنها أو يوان شيكاي أمرت ا...

🔍 Formatting validation dataset...

📝 Sample Formatted Input (first 200 chars):
Sample 1: ما الفرق بين حكومة محمد علي وعصر المماليك؟ <sep> كانت الحكومة المصرية على عهد محمد على حكومة مطلقة تسود فىها قاعدة حكم الفرد، ولكن الفارق بينها وبين ما كانت عليه فى عصر المماليك أن محمد على ب

In [7]:
# Convert to DatasetDict
datasets = DatasetDict({
    "train": Dataset.from_pandas(train_qa),
    "validation": Dataset.from_pandas(val_qa)
})

In [8]:
# Load AraBART-finetuned-wiki-ar
print("\n🔄 Loading AraBART model...")
model_name = "moussaKam/AraBART"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Add <sep> token
special_tokens = {'additional_special_tokens': ['<sep>']}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

# Verify tokenization
sample_text = "السؤال: متى تأسست السعودية؟ <sep> الجواب: 1932 <sep> السياق: تأسست المملكة..."
print("\n🔍 Tokenization Test:")
print("Tokens:", tokenizer.tokenize(sample_text)[:20])
print("<sep> token ID:", tokenizer.convert_tokens_to_ids("<sep>"))


🔄 Loading AraBART model...


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`



🔍 Tokenization Test:
Tokens: ['▁السؤال', ':', '▁متى', '▁تأسست', '▁السعودية', '؟', '<sep>', '▁الجواب', ':', '▁1932', '<sep>', '▁السياق', ':', '▁تأسست', '▁المملكة', '...']
<sep> token ID: 50002


In [9]:
# Tokenization function optimized for AraBART
def tokenize_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["answer"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    
    model_inputs["labels"] = labels["input_ids"]
    
    # # Debug sample
    # if len(model_inputs["input_ids"]) > 0:
    #         print("\n🔢 Sample Tokenized:")
    #         print("Input:", tokenizer.decode(model_inputs["input_ids"][0][:20]))
    #         print("Label:", tokenizer.decode(labels["input_ids"][0][:10]))
    
    return model_inputs

tokenized_datasets = datasets.map(tokenize_function, batched=True, batch_size=8, remove_columns=["text", "answer"])

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/550 [00:00<?, ? examples/s]

In [10]:
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    label_pad_token_id=-100
)

In [11]:
# Training arguments optimized for AraBART
training_args = Seq2SeqTrainingArguments(
    output_dir="./arabart-qa",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,  # Reduced for memory
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,
    predict_with_generate=True,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    generation_max_length=128,
    generation_num_beams=4,
    report_to="none",
    load_best_model_at_end=True
)

In [12]:
# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipykernel_19/2516391079.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [13]:
# Train the model
print("\n🚀 Starting training...")
trainer.train()


🚀 Starting training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.112700,0.100882
2,0.094000,0.096573
3,0.074800,0.098148
4,0.065500,0.097346
5,0.056400,0.101075
6,0.050000,0.102783
7,0.045100,0.103289
8,0.045700,0.105178


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will ins

TrainOutput(global_step=4704, training_loss=0.1380730972682335, metrics={'train_runtime': 2361.0114, 'train_samples_per_second': 15.932, 'train_steps_per_second': 1.992, 'total_flos': 1.1468278295691264e+16, 'train_loss': 0.1380730972682335, 'epoch': 8.0})

In [14]:
import shutil
import os

# Remove existing model directory if it exists
model_dir = "/kaggle/working/arabart_qa_model"
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

# Remove old zip file
zip_path = "/kaggle/working/arabart_qa_model.zip"
if os.path.exists(zip_path):
    os.remove(zip_path)

In [15]:
# Save model
trainer.save_model("./arabart_qa_model")
# tokenizer.save_pretrained("./arabart_qa_model")

In [16]:
# Enhanced evaluation function
def evaluate_qa(model, tokenizer, dataset, num_examples=10):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for i in range(num_examples):
        input_text = f"{dataset['question'][i]} <sep> {dataset['answer'][i]} <sep> {dataset['context'][i]}"
        
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
        
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True,
            temperature=0.9
        )
        
        print(f"\n📄 Example {i+1}:")
        print("🔍 Question:", dataset["question"][i])
        print("📜 Context:", dataset["context"][i][:100] + "...")
        print("✅ Correct:", dataset["answer"][i])
        print("🤖 Predicted:", tokenizer.decode(outputs[0], skip_special_tokens=True))
        print("-"*80)

print("\n🔍 Evaluating model...")
evaluate_qa(model, tokenizer, val_data)


🔍 Evaluating model...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



📄 Example 1:
🔍 Question: ما الفرق بين حكومة محمد علي وعصر المماليك؟
📜 Context: كانت الحكومة المصرية على عهد محمد على حكومة مطلقة تسود فىها قاعدة حكم الفرد، ولكن الفارق بينها وبين ...
✅ Correct: وجود نظام إداري منظم
🤖 Predicted: حكومة مطلقة تسود فيها قاعدة حكم الفرد
--------------------------------------------------------------------------------

📄 Example 2:
🔍 Question: ما هو الدور الإداري للوزير؟
📜 Context: شروط الوزير ومهامه شروط اختيار الوزير: ⯌العدل والمساواة والإنصاف. ⯌حسن السيرة. ⯌عفة اللسان. ⯌طهارة ا...
✅ Correct: رئيس الإدارة المركزية
🤖 Predicted: الإشراف على الضرائب وجمعها
--------------------------------------------------------------------------------

📄 Example 3:
🔍 Question: ما هو الغرض من استخدام آلات رفع الماء؟
📜 Context: علم الميكانيكا: استفاد العلماء المسلمون في علم الميكانيكا بما قدمته الأمم السابقة، ثم طوروه وأطلقوا ...
✅ Correct: لعدم إهدار الماء
🤖 Predicted: لتخزين الماء
--------------------------------------------------------------------------------

📄 Example 4:
